# Interval Tree & Sweep Line Patterns

## 1. Interval Tree

An **Interval Tree** is a balanced binary search tree designed to store intervals and efficiently answer **overlap queries**:
- **Query**: Given interval [a, b], find all intervals that overlap with it.
- **Insert/Delete**: Add or remove intervals.
- **Time complexity**: O(log n + k) where k is the number of overlapping intervals.

**Key insight**: Each node stores the **maximum endpoint** of all intervals in its subtree, enabling efficient pruning during traversal.

---

## 2. Sweep Line Algorithm

The **Sweep Line** technique processes geometric problems by imagining a vertical line sweeping across a plane from left to right. Events (start/end of intervals, points) are processed in sorted order.

**Common applications**:
- **Line Segment Intersection**: Detect intersections among multiple line segments.
- **Interval Scheduling**: Find maximum non-overlapping intervals.
- **Rectangle Union**: Compute area covered by overlapping rectangles.
- **Closest Pair**: Find closest pair of points.

**Pattern**:
1. Convert problem into **events** (start/end points).
2. **Sort events** by x-coordinate (or time).
3. **Process events** left-to-right, maintaining active set.
4. Update answer as events are processed.

---

## 3. Common Problems

### Interval Overlap Problems:
- **Meeting Rooms II**: Minimum rooms needed for overlapping meetings.
- **Merge Intervals**: Combine overlapping intervals.
- **Insert Interval**: Insert new interval into sorted list.

### Sweep Line Problems:
- **Rectangle Area**: Total area covered by rectangles.
- **Skyline Problem**: Building skyline silhouette.
- **Event Scheduler**: Maximum concurrent events.

**💡 Key Pattern**: Convert intervals to **events**, sort by time, then sweep!

In [1]:
package main

import (
	"fmt"
	"sort"
)

// Interval represents a time interval [start, end)
type Interval struct {
	Start int
	End   int
}

// IntervalNode represents a node in the interval tree
type IntervalNode struct {
	Interval Interval
	Max      int // Maximum endpoint in this subtree
	Left     *IntervalNode
	Right    *IntervalNode
}

// IntervalTree represents an interval tree for overlap queries
type IntervalTree struct {
	Root *IntervalNode
}

// NewIntervalTree creates a new interval tree
func NewIntervalTree() *IntervalTree {
	return &IntervalTree{}
}

// Insert adds an interval to the tree
func (it *IntervalTree) Insert(interval Interval) {
	it.Root = it.insertNode(it.Root, interval)
}

func (it *IntervalTree) insertNode(node *IntervalNode, interval Interval) *IntervalNode {
	// Base case: create new node
	if node == nil {
		return &IntervalNode{
			Interval: interval,
			Max:      interval.End,
		}
	}

	// Insert based on start time
	if interval.Start < node.Interval.Start {
		node.Left = it.insertNode(node.Left, interval)
	} else {
		node.Right = it.insertNode(node.Right, interval)
	}

	// Update max endpoint
	if interval.End > node.Max {
		node.Max = interval.End
	}

	return node
}

// SearchOverlap finds all intervals that overlap with the given interval
func (it *IntervalTree) SearchOverlap(interval Interval) []Interval {
	var result []Interval
	it.searchOverlapHelper(it.Root, interval, &result)
	return result
}

func (it *IntervalTree) searchOverlapHelper(node *IntervalNode, interval Interval, result *[]Interval) {
	if node == nil {
		return
	}

	// Check if current interval overlaps
	if it.doOverlap(node.Interval, interval) {
		*result = append(*result, node.Interval)
	}

	// If left subtree's max endpoint >= interval.start, there might be overlap in left
	if node.Left != nil && node.Left.Max >= interval.Start {
		it.searchOverlapHelper(node.Left, interval, result)
	}

	// Always check right subtree if current interval's start <= interval.end
	if node.Right != nil && node.Interval.Start <= interval.End {
		it.searchOverlapHelper(node.Right, interval, result)
	}
}

// doOverlap checks if two intervals overlap
func (it *IntervalTree) doOverlap(a, b Interval) bool {
	return a.Start < b.End && b.Start < a.End
}

// Event represents a sweep line event
type Event struct {
	Time     int
	Type     int // 0 = start, 1 = end
	Interval Interval
}

// SweepLine solves interval problems using sweep line algorithm
type SweepLine struct {
	Events []Event
}

// NewSweepLine creates a new sweep line solver
func NewSweepLine() *SweepLine {
	return &SweepLine{}
}

// AddInterval adds an interval to the sweep line
func (sl *SweepLine) AddInterval(interval Interval) {
	sl.Events = append(sl.Events, Event{
		Time:     interval.Start,
		Type:     0,
		Interval: interval,
	})
	sl.Events = append(sl.Events, Event{
		Time:     interval.End,
		Type:     1,
		Interval: interval,
	})
}

// MaxConcurrentIntervals finds the maximum number of overlapping intervals
func (sl *SweepLine) MaxConcurrentIntervals() int {
	if len(sl.Events) == 0 {
		return 0
	}

	// Sort events by time, with end events before start events at same time
	sort.Slice(sl.Events, func(i, j int) bool {
		if sl.Events[i].Time == sl.Events[j].Time {
			return sl.Events[i].Type > sl.Events[j].Type // end before start
		}
		return sl.Events[i].Time < sl.Events[j].Time
	})

	maxCount := 0
	currentCount := 0

	for _, event := range sl.Events {
		if event.Type == 0 { // start event
			currentCount++
			if currentCount > maxCount {
				maxCount = currentCount
			}
		} else { // end event
			currentCount--
		}
	}

	return maxCount
}

// MergeIntervals merges overlapping intervals using sweep line
func (sl *SweepLine) MergeIntervals() []Interval {
	if len(sl.Events) == 0 {
		return []Interval{}
	}

	// Convert to intervals and sort
	intervals := make([]Interval, 0, len(sl.Events)/2)
	for i := 0; i < len(sl.Events); i += 2 {
		if sl.Events[i].Type == 0 { // start event
			intervals = append(intervals, sl.Events[i].Interval)
		}
	}

	sort.Slice(intervals, func(i, j int) bool {
		return intervals[i].Start < intervals[j].Start
	})

	var merged []Interval
	for _, interval := range intervals {
		if len(merged) == 0 || merged[len(merged)-1].End < interval.Start {
			// No overlap, add new interval
			merged = append(merged, interval)
		} else {
			// Overlap, merge with last interval
			if interval.End > merged[len(merged)-1].End {
				merged[len(merged)-1].End = interval.End
			}
		}
	}

	return merged
}

// Point represents a 2D point for sweep line problems
type Point struct {
	X, Y int
}

// Rectangle represents a rectangle for area calculation
type Rectangle struct {
	X1, Y1, X2, Y2 int
}

// SweepLineGeometry handles geometric sweep line problems
type SweepLineGeometry struct {
	Rectangles []Rectangle
}

// NewSweepLineGeometry creates a new geometric sweep line solver
func NewSweepLineGeometry() *SweepLineGeometry {
	return &SweepLineGeometry{}
}

// AddRectangle adds a rectangle to the collection
func (slg *SweepLineGeometry) AddRectangle(rect Rectangle) {
	slg.Rectangles = append(slg.Rectangles, rect)
}

// RectangleAreaUnion calculates total area covered by rectangles (simplified version)
func (slg *SweepLineGeometry) RectangleAreaUnion() int {
	if len(slg.Rectangles) == 0 {
		return 0
	}

	// Create events for vertical edges
	type VerticalEvent struct {
		X      int
		Y1, Y2 int
		Type   int // 0 = left edge (start), 1 = right edge (end)
	}

	var events []VerticalEvent
	for _, rect := range slg.Rectangles {
		events = append(events, VerticalEvent{rect.X1, rect.Y1, rect.Y2, 0})
		events = append(events, VerticalEvent{rect.X2, rect.Y1, rect.Y2, 1})
	}

	// Sort events by X coordinate
	sort.Slice(events, func(i, j int) bool {
		if events[i].X == events[j].X {
			return events[i].Type < events[j].Type // start before end
		}
		return events[i].X < events[j].X
	})

	totalArea := 0
	prevX := events[0].X
	
	// Active intervals (Y-coordinates)
	var activeIntervals []Interval

	for _, event := range events {
		// Calculate area contribution from previous sweep
		if event.X > prevX {
			totalArea += (event.X - prevX) * slg.calculateActiveHeight(activeIntervals)
		}

		// Update active intervals
		if event.Type == 0 { // start event
			activeIntervals = append(activeIntervals, Interval{event.Y1, event.Y2})
		} else { // end event
			// Remove the interval
			for i, interval := range activeIntervals {
				if interval.Start == event.Y1 && interval.End == event.Y2 {
					activeIntervals = append(activeIntervals[:i], activeIntervals[i+1:]...)
					break
				}
			}
		}

		prevX = event.X
	}

	return totalArea
}

// calculateActiveHeight calculates the total height covered by active intervals
func (slg *SweepLineGeometry) calculateActiveHeight(intervals []Interval) int {
	if len(intervals) == 0 {
		return 0
	}

	// Sort intervals by start
	sort.Slice(intervals, func(i, j int) bool {
		return intervals[i].Start < intervals[j].Start
	})

	// Merge overlapping intervals and sum heights
	totalHeight := 0
	currentEnd := intervals[0].Start

	for _, interval := range intervals {
		if interval.Start > currentEnd {
			// No overlap
			totalHeight += interval.End - interval.Start
			currentEnd = interval.End
		} else if interval.End > currentEnd {
			// Partial overlap
			totalHeight += interval.End - currentEnd
			currentEnd = interval.End
		}
		// Complete overlap: no additional height
	}

	return totalHeight
}

## Examples

Below are examples showing interval tree and sweep line algorithm usage.

In [3]:
%%
// Interval Tree Example
tree := NewIntervalTree()

// Insert intervals
intervals := []Interval{
	{1, 4},
	{2, 6},
	{8, 10},
	{3, 5},
	{15, 18},
}

for _, interval := range intervals {
	tree.Insert(interval)
}

// Search for overlapping intervals
query := Interval{4, 7}
overlaps := tree.SearchOverlap(query)
fmt.Printf("Intervals overlapping with [%d, %d): ", query.Start, query.End)
for _, overlap := range overlaps {
	fmt.Printf("[%d, %d) ", overlap.Start, overlap.End)
}
fmt.Println()

Intervals overlapping with [4, 7): [2, 6) [3, 5) 


In [4]:
%%
// Sweep Line Example - Maximum Concurrent Intervals
sweep := NewSweepLine()

meetings := []Interval{
	{1, 3},
	{2, 4},
	{3, 6},
	{5, 7},
	{8, 9},
}

for _, meeting := range meetings {
	sweep.AddInterval(meeting)
}

maxRooms := sweep.MaxConcurrentIntervals()
fmt.Printf("Maximum meeting rooms needed: %d\n", maxRooms)

// Merge overlapping intervals
merged := sweep.MergeIntervals()
fmt.Print("Merged intervals: ")
for _, interval := range merged {
	fmt.Printf("[%d, %d) ", interval.Start, interval.End)
}
fmt.Println()

Maximum meeting rooms needed: 2
Merged intervals: [1, 3) [8, 9) 


In [5]:
%%
// Sweep Line Geometry Example - Rectangle Area Union
geom := NewSweepLineGeometry()

rectangles := []Rectangle{
	{1, 1, 3, 3}, // Rectangle from (1,1) to (3,3)
	{2, 2, 4, 4}, // Rectangle from (2,2) to (4,4) - overlaps with first
	{5, 1, 6, 2}, // Rectangle from (5,1) to (6,2) - separate
}

for _, rect := range rectangles {
	geom.AddRectangle(rect)
}

totalArea := geom.RectangleAreaUnion()
fmt.Printf("Total area covered by rectangles: %d\n", totalArea)

Total area covered by rectangles: 8
